In [74]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from preprocessing import preprocess

In [75]:
data = pd.read_csv("train.csv", index_col=0)
#data_plus = pd.read_csv("training_extra.csv", index_col=0)
#data = pd.concat([data, data_plus], axis=0)

In [76]:
data.tail()

,Brand,Material,Size,Compartments,Laptop Compartment,Waterproof,Style,Color,Weight Capacity (kg),Price
id,,,,,,,,,,
299995,Adidas,Leather,Small,9.0,No,No,Tote,Blue,12.730812,129.99749
299996,Jansport,Leather,Large,6.0,No,Yes,Tote,Blue,26.633182,19.85819
299997,Puma,Canvas,Large,9.0,Yes,Yes,Backpack,Pink,11.898250,111.41364
299998,Adidas,Nylon,Small,1.0,No,Yes,Tote,Pink,6.175738,115.89080
299999,Under Armour,Canvas,Small,2.0,No,Yes,Backpack,Black,18.568865,26.72762


In [77]:
data = preprocess(data)
data.head()

,Brand,Material,Size,Compartments,Laptop Compartment,Waterproof,Style,Color,Weight Capacity (kg),Price,Laptop Compartment_Missing,Waterproof_Missing,Weight Capacity (kg)_Missing
id,,,,,,,,,,,,,
0,Jansport,Leather,Medium,7.0,1.0,0.0,Tote,Black,11.611723,112.15875,0,0,0
1,Jansport,Canvas,Small,10.0,1.0,1.0,Messenger,Green,27.078537,68.88056,0,0,0
2,Under Armour,Leather,Small,2.0,1.0,0.0,Messenger,Red,16.643760,39.17320,0,0,0
3,Nike,Nylon,Small,8.0,1.0,0.0,Messenger,Green,12.937220,80.60793,0,0,0
4,Adidas,Canvas,Medium,1.0,1.0,1.0,Messenger,Green,17.749338,86.02312,0,0,0


In [78]:
data.dtypes

Brand                            object
Material                         object
Size                             object
Compartments                    float64
Laptop Compartment              float64
Waterproof                      float64
Style                            object
Color                            object
Weight Capacity (kg)            float64
Price                           float64
Laptop Compartment_Missing        int32
Waterproof_Missing                int32
Weight Capacity (kg)_Missing      int32
dtype: object

---

SANDBOX - testing before implementing in preprocessing function

In [79]:
numerical_columns = data.select_dtypes(include='number').columns
categorical_columns = data.select_dtypes(exclude='number').columns

In [ ]:
target_encoding_dict = {}
for column in categorical_columns:
    gb = data.groupby(column).agg(
        Price_mean = ("Price", "mean"),
        Count = ("Price", "count"),
        STD = ("Price", "std"),
        VAR = ("Price", "var")
    )

    for index in gb.index:
        key = f"{column}_{index}"
        mean = gb.loc[index, "Price_mean"]
        count = gb.loc[index, "Count"]
        stdeviation = gb.loc[index, "STD"]
        var = gb.loc[index, "VAR"]

        target_encoding_dict[key] = [mean, count, stdeviation, var]

    data = data.join(gb, on=column, lsuffix=f"_{column}", rsuffix="")

data.rename(columns={
    "Price_mean" : "Price_mean_Brand",
    "Count" : "Count_Brand",
    "STD" : "STD_Brand",
    "VAR" : "VAR_Brand"
}, inplace=True)

data.head()

,Brand,Material,Size,Compartments,Laptop Compartment,Waterproof,Style,Color,Weight Capacity (kg),Price,...,STD_Style,VAR_Style,Price_mean_Color,Count_Color,STD_Color,VAR_Color,Price_mean_Brand,Count_Brand,STD_Brand,VAR_Brand
id,,,,,,,,,,,,,,,,,,,,,
0,Jansport,Leather,Medium,7.0,1.0,0.0,Tote,Black,11.611723,112.15875,...,38.707882,1498.300133,81.374865,97438,39.208353,1537.294979,80.513439,46378,39.212778,1537.641978
1,Jansport,Canvas,Small,10.0,1.0,1.0,Messenger,Green,27.078537,68.88056,...,38.999298,1520.945266,81.410925,100031,38.868197,1510.736709,82.381308,46386,38.934312,1515.880618
2,Under Armour,Leather,Small,2.0,1.0,0.0,Messenger,Red,16.643760,39.17320,...,38.999298,1520.945266,81.410925,100031,38.868197,1510.736709,81.011644,47219,39.391864,1551.718941
3,Nike,Nylon,Small,8.0,1.0,0.0,Messenger,Green,12.937220,80.60793,...,38.999298,1520.945266,81.410925,100031,38.868197,1510.736709,82.381308,46386,38.934312,1515.880618
4,Adidas,Canvas,Medium,1.0,1.0,1.0,Messenger,Green,17.749338,86.02312,...,38.707882,1498.300133,81.410925,100031,38.868197,1510.736709,82.381308,46386,38.934312,1515.880618
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299995,Adidas,Leather,Small,9.0,0.0,0.0,Tote,Blue,12.730812,129.99749,...,38.999298,1520.945266,81.374865,97438,39.208353,1537.294979,82.006994,48097,38.833774,1508.062007
299996,Jansport,Leather,Large,6.0,0.0,1.0,Tote,Blue,26.633182,19.85819,...,39.332187,1547.020928,81.374865,97438,39.208353,1537.294979,82.006994,48097,38.833774,1508.062007
299997,Puma,Canvas,Large,9.0,1.0,1.0,Backpack,Pink,11.898250,111.41364,...,39.332187,1547.020928,81.402513,94561,39.039632,1524.092828,81.630864,51690,39.113769,1529.886932


In [82]:
test = pd.read_csv("test.csv", index_col=0)
test = preprocess(test)
test.head()

,Brand,Material,Size,Compartments,Laptop Compartment,Waterproof,Style,Color,Weight Capacity (kg),Laptop Compartment_Missing,Waterproof_Missing,Weight Capacity (kg)_Missing
id,,,,,,,,,,,,
300000,Puma,Leather,Small,2.0,0.0,0.0,Tote,Green,20.671147,0,0,0
300001,Nike,Canvas,Medium,7.0,0.0,1.0,Backpack,Green,13.564105,0,0,0
300002,Adidas,Canvas,Large,9.0,0.0,1.0,Messenger,Blue,11.809799,0,0,0
300003,Adidas,Nylon,Large,1.0,1.0,0.0,Messenger,Green,18.477036,0,0,0
300004,Inconnu,Nylon,Large,2.0,1.0,1.0,Tote,Black,9.907953,0,0,0


In [88]:
for key in target_encoding_dict:
    initial_column = key.split("_")[0]
    corresponding_value = key.split("_")[1]
    
    count=0
    for aggfunc in ["Price_mean", "Count", "STD", "VAR"]:
        test.loc[test[initial_column] == corresponding_value, f"{aggfunc}_{initial_column}"] = target_encoding_dict[key][count]
        count += 1

test

,Brand,Material,Size,Compartments,Laptop Compartment,Waterproof,Style,Color,Weight Capacity (kg),Laptop Compartment_Missing,...,STD_Size,VAR_Size,Price_mean_Style,Count_Style,STD_Style,VAR_Style,Price_mean_Color,Count_Color,STD_Color,VAR_Color
id,,,,,,,,,,,,,,,,,,,,,
300000,Puma,Leather,Small,2.0,0.0,0.0,Tote,Green,20.671147,0,...,38.999298,1520.945266,81.374865,97438.0,39.208353,1537.294979,82.381308,46386.0,38.934312,1515.880618
300001,Nike,Canvas,Medium,7.0,0.0,1.0,Backpack,Green,13.564105,0,...,38.707882,1498.300133,81.402513,94561.0,39.039632,1524.092828,82.381308,46386.0,38.934312,1515.880618
300002,Adidas,Canvas,Large,9.0,0.0,1.0,Messenger,Blue,11.809799,0,...,39.332187,1547.020928,81.410925,100031.0,38.868197,1510.736709,82.006994,48097.0,38.833774,1508.062007
300003,Adidas,Nylon,Large,1.0,1.0,0.0,Messenger,Green,18.477036,0,...,39.332187,1547.020928,81.410925,100031.0,38.868197,1510.736709,82.381308,46386.0,38.934312,1515.880618
300004,Inconnu,Nylon,Large,2.0,1.0,1.0,Tote,Black,9.907953,0,...,39.332187,1547.020928,81.374865,97438.0,39.208353,1537.294979,80.513439,46378.0,39.212778,1537.641978
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499995,Adidas,Canvas,Large,2.0,1.0,0.0,Messenger,Red,7.383498,0,...,39.332187,1547.020928,81.410925,100031.0,38.868197,1510.736709,81.011644,47219.0,39.391864,1551.718941
499996,Nike,Polyester,Small,9.0,0.0,1.0,Messenger,Pink,6.058394,0,...,38.999298,1520.945266,81.410925,100031.0,38.868197,1510.736709,81.630864,51690.0,39.113769,1529.886932
499997,Jansport,Nylon,Small,9.0,0.0,1.0,Tote,Green,26.890163,0,...,38.999298,1520.945266,81.374865,97438.0,39.208353,1537.294979,82.381308,46386.0,38.934312,1515.880618


In [49]:
target_encoding_dict

{'Brand_Adidas': [80.63106006158763,
  60077,
  38.805601443757126,
  1505.874703411725],
 'Brand_Inconnu': [80.87274884286451,
  9705,
  39.66340891463282,
  1573.1860067293742],
 'Brand_Jansport': [81.79127646033953,
  56076,
  38.970597560453854,
  1518.7074742188518],
 'Brand_Nike': [81.31920949891865,
  57336,
  39.129019087617884,
  1531.0801347591646],
 'Brand_Puma': [81.4486071033548,
  56814,
  39.22977038428654,
  1538.9748844038452],
 'Brand_Under Armour': [81.97631096929591,
  59992,
  38.955755789624256,
  1517.5509091408435],
 'Material_Canvas': [82.10651118478324,
  68004,
  38.997844212710774,
  1520.8318532388594],
 'Material_Inconnu': [81.63762580687671,
  8347,
  39.49083854351167,
  1559.5263288697072],
 'Material_Leather': [80.43788323498966,
  73416,
  39.09205584412581,
  1528.188830120251],
 'Material_Nylon': [81.02475983485121,
  70603,
  38.892242579660696,
  1512.6065328751724],
 'Material_Polyester': [82.03331465126209,
  79630,
  39.08646159301276,
  1527.7

In [50]:
# don't forget to update columns_to_scale

END OF THE SANDBOX

---

In [7]:
print(f"{(len(numerical_columns) + len(categorical_columns))} = {len(data.columns)}")

33 = 33


In [8]:
data.columns

Index(['Brand', 'Material', 'Size', 'Compartments', 'Laptop Compartment',
       'Waterproof', 'Style', 'Color', 'Weight Capacity (kg)', 'Price',
       'Laptop Compartment_Missing', 'Waterproof_Missing',
       'Weight Capacity (kg)_Missing', 'Price_mean_Material', 'Count_Material',
       'STD_Material', 'VAR_Material', 'Price_mean_Size', 'Count_Size',
       'STD_Size', 'VAR_Size', 'Price_mean_Style', 'Count_Style', 'STD_Style',
       'VAR_Style', 'Price_mean_Color', 'Count_Color', 'STD_Color',
       'VAR_Color', 'Price_mean', 'Count', 'STD', 'VAR'],
      dtype='object')

In [9]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import (
    OneHotEncoder,
    StandardScaler
)
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, cross_validate

In [10]:
ohe_columns = [
    "Brand",
    "Material",
    "Size",
    #"Compartments",
    "Style",
    "Color"
]

columns_to_scale = [
    "Compartments",
    "Weight Capacity (kg)"
]

In [11]:
ohe = OneHotEncoder(handle_unknown="ignore", sparse_output=False)
std = StandardScaler()

In [12]:
column_transformer = ColumnTransformer(
    transformers=[
        ("OneHotEncoding", ohe, ohe_columns),
        ("Standardization", std, columns_to_scale)
    ],
    remainder="passthrough",
    n_jobs=-1,
    verbose=3,
    verbose_feature_names_out=False
).set_output(transform="pandas")

In [13]:
pipeline = Pipeline(
    [
        ("ColumnTransformer", column_transformer),
    ]
)

In [14]:
data = pipeline.fit_transform(data)

In [15]:
data["Laptop Compartment"].unique()

array([ 1.,  0., -1.])

In [16]:
from sklearn.linear_model import LinearRegression
import xgboost as xgb

In [17]:
#model = LinearRegression()
model = xgb.XGBRegressor()

In [18]:
cv_results = cross_validate(model, data.drop(columns='Price'), data['Price'], cv=5, verbose=4, return_train_score=True, scoring='neg_root_mean_squared_error', n_jobs=-1)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:   10.2s remaining:   15.3s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   10.5s finished


In [ ]:
cv_results

{'fit_time': array([6.74164462, 6.49288511, 6.61222959, 6.49024582, 6.58041596]),
 'score_time': array([0.31557536, 0.33212972, 0.27783871, 0.3176899 , 0.36611962]),
 'test_score': array([-39.02793493, -39.23967452, -39.13886203, -39.27857476,
        -39.24026862]),
 'train_score': array([-38.14566734, -38.12717798, -38.15129002, -38.11452358,
        -38.13464684])}

In [ ]:
model.fit(data.drop(columns='Price'), data['Price'], verbose=3)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

---

DATA TEST

In [ ]:
test = pd.read_csv('test.csv', index_col=0)
test.head()

,Brand,Material,Size,Compartments,Laptop Compartment,Waterproof,Style,Color,Weight Capacity (kg)
id,,,,,,,,,
300000,Puma,Leather,Small,2.0,No,No,Tote,Green,20.671147
300001,Nike,Canvas,Medium,7.0,No,Yes,Backpack,Green,13.564105
300002,Adidas,Canvas,Large,9.0,No,Yes,Messenger,Blue,11.809799
300003,Adidas,Nylon,Large,1.0,Yes,No,Messenger,Green,18.477036
300004,NaN,Nylon,Large,2.0,Yes,Yes,Tote,Black,9.907953


In [ ]:
test = preprocess(test)
test.head()

KeyError: "Column(s) ['Price'] do not exist"

In [ ]:
test = pipeline.fit_transform(test)
test.head()

,Brand_Adidas,Brand_Inconnu,Brand_Jansport,Brand_Nike,Brand_Puma,Brand_Under Armour,Material_Canvas,Material_Inconnu,Material_Leather,Material_Nylon,...,Color_Inconnu,Color_Pink,Color_Red,Compartments,Weight Capacity (kg),Laptop Compartment,Waterproof,Laptop Compartment_Missing,Waterproof_Missing,Weight Capacity (kg)_Missing
id,,,,,,,,,,,,,,,,,,,,,
300000,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,-1.191822,0.384695,0.0,0.0,0,0,0
300001,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.539041,-0.633409,0.0,1.0,0,0,0
300002,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.231386,-0.884718,0.0,1.0,0,0,0
300003,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,-1.537995,0.070382,1.0,0.0,0,0,0
300004,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,-1.191822,-1.157162,1.0,1.0,0,0,0


In [ ]:
preds = pd.concat([pd.DataFrame(test.index), pd.DataFrame(model.predict(test))], axis=1, ignore_index=True)
preds.columns = ['id', 'Price']
print(len(preds))
preds.head()

ValueError: feature_names mismatch: ['Brand_Adidas', 'Brand_Inconnu', 'Brand_Jansport', 'Brand_Nike', 'Brand_Puma', 'Brand_Under Armour', 'Material_Canvas', 'Material_Inconnu', 'Material_Leather', 'Material_Nylon', 'Material_Polyester', 'Size_Inconnu', 'Size_Large', 'Size_Medium', 'Size_Small', 'Style_Backpack', 'Style_Inconnu', 'Style_Messenger', 'Style_Tote', 'Color_Black', 'Color_Blue', 'Color_Gray', 'Color_Green', 'Color_Inconnu', 'Color_Pink', 'Color_Red', 'Compartments', 'Weight Capacity (kg)', 'Laptop Compartment', 'Waterproof', 'Laptop Compartment_Missing', 'Waterproof_Missing', 'Weight Capacity (kg)_Missing', 'Price_meanMaterial_', 'CountMaterial_', 'STDMaterial_', 'VARMaterial_', 'Price_meanSize_', 'CountSize_', 'STDSize_', 'VARSize_', 'Price_meanStyle_', 'CountStyle_', 'STDStyle_', 'VARStyle_', 'Price_meanColor_', 'CountColor_', 'STDColor_', 'VARColor_', 'Price_mean', 'Count', 'STD', 'VAR'] ['Brand_Adidas', 'Brand_Inconnu', 'Brand_Jansport', 'Brand_Nike', 'Brand_Puma', 'Bran

In [ ]:
preds.to_csv('submission.csv', index=False)